In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from WindPy import w
from sklearn.linear_model import LinearRegression
%matplotlib inline

In [2]:
w.start()

Welcome to use Wind Quant API for Python (WindPy)!

COPYRIGHT (C) 2017 WIND INFORMATION CO., LTD. ALL RIGHTS RESERVED.
IN NO CIRCUMSTANCE SHALL WIND BE RESPONSIBLE FOR ANY DAMAGES OR LOSSES CAUSED BY USING WIND QUANT API FOR Python.


.ErrorCode=0
.Data=[OK!]

In [3]:
start_date, end_date = '2018-09-01', '2018-10-23'

# 获取股票债券指数历史收益率

In [109]:
codes = 'CBA00201.CS,000906.SH,AU9999.SGE'

In [110]:
data = w.wsd(codes, 'close', start_date, end_date)

In [111]:
df = pd.DataFrame(np.array(data.Data).T, index=data.Times, columns=data.Codes)

In [112]:
rdf = df.pct_change().dropna()

# 获取FOF基金历史收益率

In [81]:
codes = '005156.OF,005218.OF,005215.OF,005217.OF,005220.OF,005221.OF'

In [82]:
data = w.wsd(codes, 'nav', start_date, end_date)

In [83]:
fdf = pd.DataFrame(np.array(data.Data).T, index=data.Times, columns=data.Codes)

In [84]:
rfdf = fdf.pct_change().dropna()

In [113]:
print rfdf.shape
print rdf.shape

(30, 6)
(30, 3)


# 非负线性优化求解

In [121]:
TOLERANCE = 1e-10
n = rdf.shape[1]
init_weights = np.array([1. / n] * n)
X = rdf.values
y = rfdf['005156.OF'].values

In [122]:
def objective(coef, args):
    n = coef.size
    X, y = args[0], args[1]
    target = np.dot(X, coef)
#     error = np.sum(np.square(target - y))
    error = np.sum(np.square(target - y)) + np.linalg.norm(coef, 1) # lasso error
    return error / n

In [123]:
constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1.0},
               {'type': 'ineq', 'fun': lambda x: x})

In [124]:
opt_res = minimize(fun=objective,
                   x0=init_weights,
                   args=[X, y],
                   constraints=constraints,
                   tol=TOLERANCE,
                   options={'disp': True})

Optimization terminated successfully.    (Exit mode 0)
            Current function value: 0.333349093304
            Iterations: 13
            Function evaluations: 65
            Gradient evaluations: 13


In [125]:
opt_res.x

array([0.81438373, 0.10851891, 0.07709736])